<h2><i>kiara</i>: Network Analysis</h2>

Welcome back! Now that we're comfortable with what <i>kiara</i> looks like and what it can do to help track your data and your research process, let's try out some of the digital analysis tools, starting with <b>Network Analysis</b>.

<h2>Why Network Analysis?</h2>

Network Analysis offers a computational and quantitative means to examine and explore relational objects, with proxies to measure structural roles and concepts such as power and influence. Doing so digitally - and at scale - also allows us to consider these kinds of questions with large amounts of material or documents that was not  heretofore manageable with qualitative or manual approaches.

>INSERT REFERENCES OR WIDER REFERENCING HERE: PROGRAMMING HISTORIAN LINKS W/ BIBLIOGRAPHY SECTION BELOW

<h3>Getting Started</h3>

Let's start by double checking that we have all the required plugins and setting up an API for us to use <i>kiara</i>. We'll do this all in one go this time, but if you're unsure, feel free to head back to the <a href="http://dharpa.org/kiara.documentation/latest/workshop/workshop/">installation notebook</a> to look over this section again.

In [1]:
import csv
import networkx as nx
import matplotlib.pyplot as plt

try:
    from kiara_plugin.jupyter import ensure_kiara_plugins
except:
    import sys
    print("Installing 'kiara_plugin.jupyter'...")
    !{sys.executable} -m pip install -q kiara_plugin.jupyter
    from kiara_plugin.jupyter import ensure_kiara_plugins

ensure_kiara_plugins()

from kiara.api import KiaraAPI
kiara = KiaraAPI.instance()

Great, we're all set up. Let's download some sample data for our network analysis: this is some epistolary data taken from the <i>State Papers of England</i> between 1534-1540. 

<span style="color:blue">insert links for references when needed</span> 

As before, we can use the <i>kiara</i> function `download.file` to access our material, specifying the <span style="color:green">inputs</span> and runnning the function as our <span style="color:red">outputs</span>.

In [2]:
correspsearch = kiara.run_job('import.local.file', inputs={'path':'correspSearch.csv'})
correspsearch

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field   value                                                                                                                          │
│  ─────────────────────────────────────────────────────────────────────────────────────                                                   │
│   file    Source,Target                                                                                                                  │
│           "Wagner, M.W.","Trew, Christoph Jacob"                                                                                         │
│           "Basellius, Nikolaus","Melanchthon, Philipp"                                                                                   │
│           "Marx, Karl F. H.",Jean Paul                                                                                                   │
│           Isacco Artom,Alfred Escher                                                                                                     │
│           Isacco Artom,Alfred Escher                                                                                                     │
│           Edoardo de Launay,Alfred Escher                                                                                                │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Wedeke, Johann Christoph","Schleiermacher, Friedrich"                                                                         │
│           "Hoffmann, Johann Leonhard (1710-1782)","Haller, Albrecht von"                                                                 │
│           "Hoffmann, Johann Leonhard (1710-1782)","Haller, Albrecht von"                                                                 │
│           "Stupanus, Johann Niklaus","Camerarius, Joachim"                                                                               │
│           "Stupanus, Johann Niklaus","Camerarius, Joachim"                                                                               │
│           "Stupanus, Johann Niklaus","Herden, Balthasar von"                                                                             │
│           Jakob Benedikt Schmid,Alfred Escher     

<h2>Creating a Network</h2>

Time to make our network from this data. Let's start by searching for the <i>kiara</i> modules that are included in the `kiara_plugin.network_analysis` package.

In [4]:
infos = metadata = kiara.retrieve_operations_info()
operations = {}
for op_id, info in infos.item_infos.items():
    if info.context.labels.get("package", None) == "kiara_plugin.network_analysis":
        operations[op_id] = info

print(operations.keys())

dict_keys(['create.network_data.from.tables', 'export.network_data.as.csv_files', 'export.network_data.as.graphml_file', 'export.network_data.as.sql_dump', 'export.network_data.as.sqlite_db', 'network_data.check_clusters'])


There's lots of options for analysis, but we want to make our network first. Let's have a look what we need with the function `create.network_data.from.tables` using `kiara.retrieve_operation_info` once more.

<span style="color:blue">this will make more sense/have more options when analysis modules are incorporated into the network plugin</span>

In [5]:
kiara.retrieve_operation_info('create.network_data.from.tables')

Author(s)                                                                                                                                  
                     Lena Jaskov         helena.jaskov@uni.lu                                                                               
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         network_analysis                                                                                          
                     Labels       package: kiara_plugin.network_analysis                                                                    
                     References   source_repo: ]8;id=527211;https://github.com/DHARPA-Project/kiara_plugin.network_analysis\https://github.com/DHARPA-Project/kiara_plugin.network_analysis]8;;\                              
                                  documentation: ]8;id=148450;https://DHARPA-Project.github.io/kiara_plugin.network_analysis/\https://DHARPA-Project.github.io/kiara_plugin.network_analysis/]8;;\                            
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Create a graph object from one or two tables.                                                          
                                                                                                                                            
                                     This module needs at least one table as input, providing the edges of the resulting network data       
                                     set.                                                                                                   
                                     If no further table is created, basic node information will be automatically created by using unique   
                                     values from                                                                                            
                                     the edges source and target columns.                                                                   
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name           type     description                            Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       edges                table    A table that contains the edges        yes        -- no default --     
                                                                     data.                                                                  
                                       source_column_name   string   The name of the source column name     no         source               
                                                                     in the edges table.                                                    
                                       target_column_name   string   The name of the target column name     no         target               
                      

Like other network analysis tools, <i>kiara</i> first needs the data as an edge table. This means we first have to transform the data we downloaded earlier into a table before we can create the network data. Let's start by using the `create.table.from.file` function that we used in the first notebook and storing this as our <b>edges</b>, then use this to create our network data using the `create.network_data.from.tables` that we just read about. In this, we are defining two different sets of <span style="color:green">inputs</span>, overriding the first variable once we have used it to create our table.

We're going to be using the network data quite a lot in the rest of the notebook, so lets store the output for this function as <b>network_data</b>. We can import a separate table with the nodes in, but this is optional, and for the moment let's stick with just the edge table.

In [6]:
inputs = {
    "file": correspsearch['file']
}

outputs = kiara.run_job('create.table.from.file', inputs={'file': correspsearch['file']})

edges = outputs['table']

inputs = {
    'edges': edges,
    'source_column_name': 'f0',
    'target_column_name': 'f1'
}

correspsearch = kiara.run_job('create.network_data.from.tables', inputs=inputs)
correspsearch

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field          value                                                                                                                   │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────                                  │
│   network_data                                                                                                                           │
│                  Table: edges                                                                                                            │
│                                                                                                                                          │
│                    source                                    target                                                                      │
│                   ───────────────────────────────────────────────────────────────────────                                                │
│                    Source                                    Target                                                                      │
│                    Wagner, M.W.                              Trew, Christoph Jacob                                                       │
│                    Basellius, Nikolaus                       Melanchthon, Philipp                                                        │
│                    Marx, Karl F. H.                          Jean Paul                                                                   │
│                    Isacco Artom                              Alfred Escher                                                               │
│                    Isacco Artom                              Alfred Escher                                                               │
│                    Edoardo de Launay                         Alfred Escher                                                               │
│                    Wedeke, Johann Christoph                  Schleiermacher, Friedrich                                                   │
│                    Wedeke, Johann Christoph                  Schleiermacher, Friedrich                                                   │
│                    Wedeke, Johann Christoph                  Schleiermacher, Friedrich                                                   │
│                    Wedeke, Johann Christoph                  Schleiermacher, Friedrich                                                   │
│                    Wedeke, Johann Christoph                  Schleiermacher, Friedrich                                                   │
│                    Wedeke, Johann Christoph                  Schleiermacher, Friedrich                                                   │
│                    Wedeke, Johann Christoph                  Schleiermacher, Friedrich                                                   │
│                    Wedeke, Johann Christoph                  Schleiermacher, Friedrich                                                   │
│                    Wedeke, Johann Christoph                  Schleiermacher, Friedrich                                                   │
│                    ...                                       ...                                                                         │
│                    ...                                       ...                                                                         │
│                    Peters                                    Spohr, Louis                                                                │
│                    Peters                         

<h2>Network Info</h2>

In [7]:
correspsearch = correspsearch['network_data']
correspsearch.get_property_data('metadata.graph_properties')

 number_of_nodes            16523                                
  properties_by_graph_type   {                                    
                               "graph_type": "directed",          
                               "number_of_edges": 24249           
                             }                                    
                             {                                    
                               "graph_type": "undirected",        
                               "number_of_edges": 20325           
                             }                                    
                             {                                    
                               "graph_type": "directed-multi",    
                               "number_of_edges": 130121          
                             }                                    
                             {                                    
                               "graph_type": "undirected-multi",  
                               "number_of_edges": 130121          
                             }                                    
  number_of_self_loops       111                                  
  number_of_parallel_edges   105872

In [20]:
kiara.retrieve_operation_info('network_data.check_clusters')

Author(s)                                                                                                                                  
                     Lena Jaskov         helena.jaskov@uni.lu                                                                               
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         network_analysis                                                                                          
                     Labels       package: kiara_plugin.network_analysis                                                                    
                     References   source_repo: ]8;id=348919;https://github.com/DHARPA-Project/kiara_plugin.network_analysis\https://github.com/DHARPA-Project/kiara_plugin.network_analysis]8;;\                              
                                  documentation: ]8;id=976060;https://DHARPA-Project.github.io/kiara_plugin.network_analysis/\https://DHARPA-Project.github.io/kiara_plugin.network_analysis/]8;;\                            
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Analyse clustering in network data.                                                                    
                                                                                                                                            
                                     This module analyses network data and checks if it contains clusters, and if so, how many.             
                                     It also extracts the largest component of the graph.                                                   
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name     type           description                            Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       network_data   network_data   The network data to analyze.           yes        -- no default --     
                                                                                                                                            
                                                                                                                                            
                     Outputs                                                                                                                
                                       field name             type           description                                                    
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       largest_component      network_data   A sub-graph of the largest component of the graph.             
                                       number_of_components   integer        The number of components in the graph.                         
                      

In [30]:
output = kiara.run_job('network_data.check_clusters', inputs={'network_data':correspsearch})
output

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field                  value                                                                                                           │
│  ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────            │
│   is_connected           False                                                                                                           │
│   largest_component                                                                                                                      │
│                          Table: edges                                                                                                    │
│                                                                                                                                          │
│                            source                                          target                                                        │
│                           ─────────────────────────────────────────────────────────────────────────────────────────                      │
│                            Isacco Artom                                    Alfred Escher                                                 │
│                            Samuel Schwarz                                  Alfred Escher                                                 │
│                            Ruysch, Frederik                                Volckamer, Johann Georg                                       │
│                            Ruysch, Frederik                                Heister, Lorenz                                               │
│                            Breyer (Bürgermeister der Stadt Baden), Franz   Rollett, Alexander                                            │
│                            Bahr, Hermann                                   Trebitsch, Siegfried                                          │
│                            Bahr, Hermann                                   Hofmannsthal, Hugo von                                        │
│                            Bahr, Hermann                                   Salten, Felix                                                 │
│                            Bahr, Hermann                                   Block, Paul                                                   │
│                            Bahr, Hermann                                   Epstein, Stephan                                              │
│                            Bahr, Hermann                                   Schmidt, Erich                                                │
│                            Bahr, Hermann                                   Beer-Hofmann, Richard                                         │
│                            Bahr, Hermann                                   Schnitzler, Julius                                            │
│                            Bahr, Hermann                                   Redlich, Josef                                                │
│                            Bahr, Hermann                                   Schlesinger, Fritz                                            │
│                            Bahr, Hermann                                   Fred, W.                                                      │
│                            ...                                             ...                                                           │
│                            ...                                             ...                                                           │
│                            Räte Hz. Johann Friedri

In [31]:
network_data = output['largest_component']

network_data.get_property_data('metadata.graph_properties')

 number_of_nodes            15812                                
  properties_by_graph_type   {                                    
                               "graph_type": "directed",          
                               "number_of_edges": 19900           
                             }                                    
                             {                                    
                               "graph_type": "undirected",        
                               "number_of_edges": 19900           
                             }                                    
                             {                                    
                               "graph_type": "directed-multi",    
                               "number_of_edges": 19900           
                             }                                    
                             {                                    
                               "graph_type": "undirected-multi",  
                               "number_of_edges": 19900           
                             }                                    
  number_of_self_loops       21                                   
  number_of_parallel_edges   0

<h2>Onboarding Data: An Alternative</h2>

In [25]:
kiara.retrieve_operation_info('onboard.gml_file')

Author(s)                                                                                                                                  
                     Lena Jaskov                helena.jaskov@uni.lu                                                                        
                     Mariella De Crouy Chanel   mariella.decrouychanel@uni.lu                                                               
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         playground                                                                                                
                     Labels       package: kiara_plugin.playground                                                                          
                     References   source_repo: ]8;id=144747;https://github.com/DHARPA-Project/kiara_plugin.playground\https://github.com/DHARPA-Project/kiara_plugin.playground]8;;\                                    
                                  documentation: ]8;id=756820;https://DHARPA-Project.github.io/kiara_plugin.playground/\https://DHARPA-Project.github.io/kiara_plugin.playground/]8;;\                                  
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   This is a preliminary module for onboarding network data from gml files. It will likely be replaced    
                                     by more generic onboarding modules when those are ready.                                               
                                                                                                                                            
                                     Based on networkX deserialise GML file method:                                                         
                                     https://networkx.org/documentation/stable/reference/readwrite/generated/networkx.readwrite.gml.read…   
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name   type     description                                    Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       path         string   The path to the local file.                    yes        -- no default --     
                                       label        string   The node attribute that holds the 'label'      no         label                
                                                             information. Set this input to 'id' when                                       
                                                             there is no 'label' attribute in gml file.                                     
                                                                                                                                            
                                                                                                                                            
                     Outputs                                                                                                                
                                  

In [26]:
lesmis = kiara.run_job('onboard.gml_file', inputs={'path':'lesmis.gml'})
lesmis

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field          value                                                                                                                   │
│  ────────────────────────────────────────────────────────────                                                                            │
│   network_data                                                                                                                           │
│                  Table: edges                                                                                                            │
│                                                                                                                                          │
│                    source           target           value                                                                               │
│                   ─────────────────────────────────────────                                                                              │
│                    Myriel           Napoleon         1                                                                                   │
│                    Myriel           MlleBaptistine   8                                                                                   │
│                    Myriel           MmeMagloire      10                                                                                  │
│                    Myriel           CountessDeLo     1                                                                                   │
│                    Myriel           Geborand         1                                                                                   │
│                    Myriel           Champtercier     1                                                                                   │
│                    Myriel           Cravatte         1                                                                                   │
│                    Myriel           Count            2                                                                                   │
│                    Myriel           OldMan           1                                                                                   │
│                    Myriel           Valjean          5                                                                                   │
│                    MlleBaptistine   MmeMagloire      6                                                                                   │
│                    MlleBaptistine   Valjean          3                                                                                   │
│                    MmeMagloire      Valjean          3                                                                                   │
│                    Labarre          Valjean          1                                                                                   │
│                    Valjean          Marguerite       1                                                                                   │
│                    Valjean          MmeDeR           1                                                                                   │
│                    ...              ...              ...                                                                                 │
│                    ...              ...              ...                                                                                 │
│                    Bossuet          Grantaire        3                                                                                   │
│                    Bossuet          MmeHucheloup  

<h2>Network Analysis: Statistical Measures</h2>

In [27]:
kiara.retrieve_operation_info('create.degree_rank_list')

Author(s)                                                                                                                                  
                     Caitlin Burge              caitlin.burge@uni.lu                                                                        
                     Mariella De Crouy Chanel   mariella.decrouychanel@uni.lu                                                               
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         playground                                                                                                
                     Labels       package: kiara_plugin.playground                                                                          
                     References   source_repo: ]8;id=80646;https://github.com/DHARPA-Project/kiara_plugin.playground\https://github.com/DHARPA-Project/kiara_plugin.playground]8;;\                                    
                                  documentation: ]8;id=172666;https://DHARPA-Project.github.io/kiara_plugin.playground/\https://DHARPA-Project.github.io/kiara_plugin.playground/]8;;\                                  
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Creates an ordered table with the rank and raw score for degree centrality.                            
                                                                                                                                            
                                     In an undirected graph, degree centrality measures the number of independent connections each node     
                                     has.                                                                                                   
                                                                                                                                            
                                     Uses networkx degree.                                                                                  
                                     https://networkx.org/documentation/stable/reference/generated/networkx.classes.function.degree.html    
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name           type           description                      Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       network_data         network_data   The network graph being          yes        -- no default --     
                                                                           queried.                                                         
                                       weighted_degree      boolean        Boolean to indicate whether to   no         True                 
                                                                           calculate weighted degree or                                     
                                                                           not. Conditions for                                              
                                   

In [29]:
output = kiara.run_job('create.degree_rank_list', inputs={'network_data':network_data})
output

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field                value                                                                                                             │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   centrality_network                                                                                                                     │
│                        Table: edges                                                                                                      │
│                                                                                                                                          │
│                          source                                          target                                                          │
│                         ─────────────────────────────────────────────────────────────────────────────────────────                        │
│                          Isacco Artom                                    Alfred Escher                                                   │
│                          Samuel Schwarz                                  Alfred Escher                                                   │
│                          Ruysch, Frederik                                Volckamer, Johann Georg                                         │
│                          Ruysch, Frederik                                Heister, Lorenz                                                 │
│                          Breyer (Bürgermeister der Stadt Baden), Franz   Rollett, Alexander                                              │
│                          Bahr, Hermann                                   Trebitsch, Siegfried                                            │
│                          Bahr, Hermann                                   Hofmannsthal, Hugo von                                          │
│                          Bahr, Hermann                                   Salten, Felix                                                   │
│                          Bahr, Hermann                                   Block, Paul                                                     │
│                          Bahr, Hermann                                   Epstein, Stephan                                                │
│                          Bahr, Hermann                                   Schmidt, Erich                                                  │
│                          Bahr, Hermann                                   Beer-Hofmann, Richard                                           │
│                          Bahr, Hermann                                   Schnitzler, Julius                                              │
│                          Bahr, Hermann                                   Redlich, Josef                                                  │
│                          Bahr, Hermann                                   Schlesinger, Fritz                                              │
│                          Bahr, Hermann                                   Fred, W.                                                        │
│                          ...                                             ...                                                             │
│                          ...                                             ...                                                             │
│                          Räte Hz. Johann Friedrichs d. M.                Räte Kf. Moritz’                                                │
│                          hzl. Statthalter und Räte

In [ ]:
kiara.retrieve_operation_info('create.betweenness_rank_list')

In [ ]:
centrality_network = output['centrality_network']

output = kiara.run_job('create.betweenness_rank_list', inputs={'network_data':centrality_network, 'weight_column_name':'value', 'weighted_betweenness':True})

output

In [ ]:
kiara.retrieve_operation_info('create.eigenvector_rank_list')

In [ ]:
centrality_network = output['centrality_network']

output = kiara.run_job('create.eigenvector_rank_list', inputs={'network_data':centrality_network, 'weight_column_name':'value'})

output

In [ ]:
kiara.retrieve_operation_info('compute.modularity_group')

In [ ]:
network = output['centrality_network']

output = kiara.run_job('compute.modularity_group', inputs={'network_data':network})

output

In [ ]:
kiara.retrieve_operation_info('create.cut_point_list')

In [ ]:
centrality_network = output['centrality_network']

output = kiara.run_job('create.cut_point_list', inputs={'network_data':centrality_network})

output

In [ ]:
output['centrality_network'].lineage

In [ ]:
kiara.retrieve_operation_info('export.network_data.as.graphml_file')